# Data Engineer Challenge

### Instalación de liberías necesarias

In [1]:
!pip install memory-profiler
!pip install line_profiler
!pip install emoji
!pip install pandas
!pip install pytest

### Importación y setup

In [2]:
%load_ext memory_profiler
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [3]:
from q1_memory import q1_memory
from q2_memory import q2_memory
from q3_memory import q3_memory
from q1_time import q1_time
from q2_time import q2_time
from q3_time import q3_time
from pandas import DataFrame as visualizador
from etils import ecolab

### Definición de constantes

In [4]:
FILE_PATH = "../data/farmers-protest-tweets-2021-2-4.json"

### ✅ Supuestos realizados

- Se asumió que el campo de fecha está en `"date"` en formato ISO 8601.
- El nombre de usuario se encuentra en `"user.username"`.
- Registros con campos faltantes fueron ignorados.
---

# Pregunta 1 - Tweets por Fecha y Usuario más Activo
> **Objetivo:** Obtener las 10 fechas con más tweets y, para cada una de esas fechas, el usuario más activo (con más publicaciones).

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, implementa las siguientes estrategias:

- Procesa el archivo en modo **streaming** (línea por línea), evitando cargar todo el contenido en memoria.
- Utiliza un **heap limitado** (`min-heap`) para mantener únicamente las **10 fechas con más tweets**, descartando automáticamente las menos relevantes.
- Evita almacenar conteos de todas las fechas posibles, lo que reduce significativamente el uso de memoria frente a otras alternativas.
- Solo conserva información de conteo de usuarios por fecha, necesaria para determinar el usuario más activo en cada una de las 10 fechas seleccionadas.
- Ignora registros incompletos (sin fecha o sin username).
- Convierte las fechas a tipo `datetime.date` solo al final, una vez determinadas las fechas relevantes.


#### Prueba de memoria

In [5]:
%memit q1_memory(FILE_PATH)

peak memory: 114.16 MiB, increment: 6.31 MiB


#### Análisis de memoria

In [6]:
# %mprun -f q1_memory q1_memory(FILE_PATH)  # resultado en la carpeta outs

#### Prueba de tiempo

In [7]:
%timeit -n 10 -r 3 q1_memory(FILE_PATH)

3.45 s ± 119 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Resultado

In [8]:
fecha_y_usuarios = q1_memory(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** de tweets completamente en memoria, reduciendo el tiempo asociado a operaciones de lectura desde disco (I/O).
- Emplea estructuras eficientes (`Counter` y `defaultdict`) para realizar conteos y agrupaciones rápidamente en memoria.
- Evita validaciones o transformaciones costosas dentro del loop principal, procesando los datos de forma directa.
- Realiza las conversiones de strings a `datetime.date` **solamente al final**, una vez que ya se han identificado las 10 fechas con más tweets.
- Ignora registros incompletos (sin fecha o sin username) para evitar errores o procesamiento innecesario.


#### Prueba de memoria

In [9]:
%memit q1_time(FILE_PATH)

peak memory: 526.34 MiB, increment: 412.57 MiB


#### Prueba de tiempo

In [10]:
%timeit -n 10 -r 3 q1_time(FILE_PATH)

3.16 s ± 97.4 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [11]:
# %lprun -f q1_time q1_time(FILE_PATH) # resultado en la carpeta outs

#### Resultado

In [12]:
fecha_y_usuarios = q1_time(FILE_PATH)
visualizador(fecha_y_usuarios, columns=["Fecha", "Usuarios"])

,Fecha,Usuarios
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,neetuanjle_nitu
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


---
# Pregunta 2 - Los top 10 emojis más usados
> **Objetivo:** Obtener los 10 emojis más usados con su respectivo conteo

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo de tweets en modo **streaming** (línea por línea), evitando cargar el archivo completo en memoria.
- Analiza únicamente el campo de texto (`content`) de cada tweet, sin almacenar los textos ni listas completas.
- Utiliza estructuras simples y eficientes (diccionario plano) para contar ocurrencias de emojis.
- Recorre el texto carácter por carácter para identificar emojis de forma directa, sin operaciones costosas adicionales.
- Solo mantiene en memoria el diccionario de conteo acumulado de emojis, que tiende a ser pequeño.
- Ordena y extrae los 10 emojis más frecuentes una vez finalizado el procesamiento completo.

#### Prueba de memoria

In [13]:
%memit q2_memory(FILE_PATH)

peak memory: 293.40 MiB, increment: 0.00 MiB


#### Análisis de memoria

In [14]:
# %mprun -f q2_memory q2_memory(FILE_PATH) # resultado en la carpeta outs 

#### Prueba de tiempo

In [15]:
%timeit -n 10 -r 3 q2_memory(FILE_PATH)

5.64 s ± 95 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [16]:
emojis_y_conteo = q2_memory(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108


## Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Carga **todo el archivo** en memoria utilizando `readlines()`, lo que reduce el tiempo de acceso al disco (I/O).
- Utiliza la estructura `set` construida a partir de `emoji.EMOJI_DATA.keys()` para identificar rápidamente si un carácter es un emoji, con operaciones de búsqueda en tiempo constante.
- Emplea `Counter` de la librería estándar para acumular los conteos de emojis de forma eficiente y rápida.
- Recorre el texto carácter por carácter, sin validaciones adicionales innecesarias dentro del bucle.
- Ignora líneas con errores de decodificación y tweets sin campo `content` para evitar procesamiento extra.
- Extrae los 10 emojis más frecuentes utilizando el método `most_common()` de `Counter`, que es altamente optimizado.


#### Prueba de memoria

In [17]:
%memit q2_time(FILE_PATH)

peak memory: 529.20 MiB, increment: 237.19 MiB


#### Prueba de tiempo

In [18]:
%timeit -n 10 -r 3 q2_time(FILE_PATH)

4.05 s ± 98.6 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [19]:
# %lprun -f q2_time q2_time(FILE_PATH) # resultado en la carpeta outs

#### Respuesta

In [20]:
emojis_y_conteo = q2_time(FILE_PATH)
visualizador(emojis_y_conteo, columns=["Emojis", "Conteos"])

,Emojis,Conteos
0,🙏,7286
1,😂,3072
2,🚜,2972
3,✊,2411
4,🌾,2363
5,🏻,2080
6,❤,1779
7,🤣,1668
8,🏽,1218
9,👇,1108


---
# Pregunta 3 - Los top 10 usuarios más influyentes
> **Objetivo:** Obtener el top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

## Enfoque de memoria optimizada

Para minimizar el uso de memoria, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo **línea por línea** (modo streaming), evitando cargar todo el contenido en memoria.
- Utiliza una función generadora que produce **uno por uno** los nombres de usuario mencionados en cada tweet.
- Solo almacena en memoria los conteos acumulados por usuario utilizando la estructura `Counter`, que es eficiente para este tipo de operaciones.
- Ignora los tweets mal formateados o sin campo `mentionedUsers`, evitando validaciones y estructuras adicionales.
- Devuelve los 10 usuarios más mencionados ordenados por frecuencia, usando el método `most_common()` de `Counter`.

#### Prueba de memoria

In [21]:
%memit q3_memory(FILE_PATH)

peak memory: 446.57 MiB, increment: 0.01 MiB


#### Análisis de memoria

In [22]:
# %mprun -f q3_memory q3_memory(FILE_PATH)  # resultado en la carpeta outs

#### Prueba de tiempo 

In [23]:
%timeit -n 10 -r 3 q3_memory(FILE_PATH)

3.3 s ± 37.9 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Respuesta

In [24]:
usuarios_mas_influyentes = q3_memory(FILE_PATH)
visualizador(usuarios_mas_influyentes, columns=["Usuario", "Menciones"])

,Usuario,Menciones
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926


### Enfoque de tiempo optimizado

Para minimizar el tiempo de ejecución, esta versión de la función implementa las siguientes estrategias:

- Procesa el archivo línea por línea (modo streaming).
- Utiliza una expresión regular para **filtrar** solo las líneas que contienen al menos una mención válida en el campo `mentionedUsers`, reduciendo significativamente el número de parseos JSON necesarios.
- Solo parsea los tweets que cumplen con ese patrón, lo que mejora el rendimiento en archivos con muchos tweets sin menciones.
- Utiliza la estructura `Counter` para contar eficientemente la cantidad de veces que cada usuario es mencionado.
- Ignora los tweets mal formateados o sin usuarios mencionados, evitando validaciones adicionales o estructuras innecesarias.
- Devuelve los 10 usuarios más mencionados ordenados por frecuencia, utilizando el método `most_common()` de `Counter`.


#### Prueba de memoria

In [25]:
%memit q3_time(FILE_PATH)

peak memory: 446.59 MiB, increment: 0.00 MiB


#### Prueba de tiempo

In [26]:
%timeit -n 10 -r 3 q3_time(FILE_PATH)

2.13 s ± 27.2 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


#### Análisis de tiempo

In [27]:
# %lprun -f q3_time q3_time(FILE_PATH) # resultado en la carpeta outs 

#### Resultado

In [28]:
usuarios_mas_influyentes = q3_time(FILE_PATH)
visualizador(usuarios_mas_influyentes, columns=["Usuario", "Menciones"])

,Usuario,Menciones
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926


--- 
### ✅ Pruebas unitarias

Las funciones `q1_memory`, `q1_time`, `q2_memory`, `q2_time`, `q3_memory` y `q3_time` fueron validadas mediante pruebas unitarias ubicadas en la carpeta `tests/`.

Las pruebas cubren los siguientes aspectos clave:

- **Formato de salida:** Verifica que cada función devuelva los datos en el formato esperado (ej. lista de tuplas con tipos correctos).
- **Consistencia entre versiones:** Asegura que las funciones optimizadas para tiempo y memoria generen el mismo resultado para un input controlado.
- **Robustez básica:** Evalúa el comportamiento ante estructuras comunes (ej. emojis, menciones y fechas válidas).

Estas pruebas pueden ejecutarse directamente desde el notebook.


In [29]:
!pytest ../tests

============================= test session starts =============================
platform win32 -- Python 3.10.16, pytest-8.3.5, pluggy-1.5.0
rootdir: C:\Users\and88\OneDrive\Documents\Python\latam_challenge_DE
plugins: anyio-4.9.0
collected 6 items

..\tests\test_q1.py ..                                                   [ 33%]
..\tests\test_q2.py ..                                                   [ 66%]
..\tests\test_q3.py ..                                                   [100%]

============================== 6 passed in 0.08s ==============================
